In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import matplotlib.pylab as pll
from collections import Counter
from datetime import datetime
import time
import random
import math
import os
from sklearn import metrics
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import hdbscan
import colorcet as cc
import kaleido

In [ ]:
slurm1 = pd.read_csv('SlurmPlot2.csv')
slurm1.tail(100)

In [ ]:
slurm1['year'] = pd.DatetimeIndex(slurm1['Submit']).year
slurm1['month'] = pd.DatetimeIndex(slurm1['Submit']).month
slurm1['year'] = slurm1['year'].apply(str)
slurm1['month'] = slurm1['month'].apply(str)
slurm1['JobID'] = slurm1['JobID'].astype(str)
slurm1['yearmonth'] = slurm1['year'] + '-' + slurm1['month']
slurm1 = slurm1[slurm1["Starttimestamp"] != 'Unknown']
slurm1 = slurm1[slurm1["Endtimestamp"] != 'Unknown']
slurm1['JobLen'] = slurm1['Endtimestamp'].astype(float) - slurm1['Starttimestamp'].astype(float)
slurm1['Totalres'] = slurm1['JobLen'] * slurm1['NNodes']
discard = ["."]
slurm = slurm1[~slurm1.JobID.str.contains('|'.join(discard),regex = False)]

In [ ]:
df_list = []
def function1(years):
    for i in years:
        temp = slurm[slurm.yearmonth.str.contains(i,regex = False)]
        df_list.append(temp)

In [ ]:
years = ['2019','2020','2021','2022']
function1(years)

In [ ]:
for c in years:
    exec('year_{} = pd.DataFrame()'.format(c))
year_2019 = df_list[0]
year_2020 = df_list[1]
year_2021 = df_list[2]
year_2022 = df_list[3]
Year_2019 = year_2019.sample(frac = 0.10)
Year_2020 = year_2020.sample(frac = 0.15)
Year_2021 = year_2021.sample(frac = 0.15)
Year_2022 = year_2022.sample(frac = 0.01)       

In [ ]:
sample = pd.concat([Year_2019, Year_2020,Year_2021,Year_2022])
cluster = sample.sample(n = 50000)


In [ ]:
ClusterData = cluster[['JobID','NNodes','TimelimitRaw']]
ax = sns.pairplot(ClusterData)
if not os.path.exists('Clustering_par1'):
    os.makedirs('Clustering_par1')
plt.savefig('Clustering_par1/pairplot1.png',bbox_inches='tight', dpi=100)
plt.show()

# Cubic root scale

In [ ]:
ClusterData.head()

In [ ]:
# ClusterDataa = ClusterData.drop(['Clustered','Dendcluster','DBcluster'], axis=1)

In [ ]:
ClusterDataa2 = ClusterData.copy()

In [ ]:
ClusterDataa2['NNodes'] = np.power(ClusterData['NNodes'],(1/2))
ClusterDataa2['TimelimitRaw'] = np.power(ClusterData['TimelimitRaw'],(1/2))

In [ ]:
#Dropping the JobID column to keep only numerical values to cluster
ClusterDataacs = ClusterDataa2.drop(['JobID'], axis=1)
sc_x = StandardScaler()
ClusterDataacs = sc_x.fit_transform(ClusterDataacs)

## K-Means

In [ ]:
#K-Means Clustering Algorithm
n_clusters = 10
kmeanss = KMeans(n_clusters=n_clusters)
kmeanss.fit(ClusterDataacs)
ClusterData['Clustered'] = pd.Series(kmeanss.labels_, index=ClusterData.index)
ClusterData['Clustered'].value_counts()

In [ ]:

plt.figure(figsize=(15, 5))
fig = px.scatter(ClusterData, x='NNodes', y='TimelimitRaw', color='Clustered')
fig.show()

In [ ]:
plt.figure(figsize=(15, 5))
sns.scatterplot(x='NNodes', y='TimelimitRaw', hue='Clustered',data=ClusterData,palette="bright")

## Standard Scaling

In [ ]:
#Dropping the JobID column to keep only numerical values to cluster
ClusterData1s = ClusterData.drop(['JobID'], axis=1)
sc_x = StandardScaler()
ClusterData1s = sc_x.fit_transform(ClusterData1s)

In [ ]:
ClusterData1s = ClusterData.drop(['Clustered','DBcluster','JobID'], axis=1)

In [ ]:
ClusterData.head(1)

In [ ]:
sc_x = StandardScaler()
ClusterData1s = sc_x.fit_transform(ClusterData1s)

# K - Means

In [ ]:

#Elbow Method to determine the best K value
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(ClusterData1s)
    distortions.append(kmeanModel.inertia_)
plt.figure(figsize=(12,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
if not os.path.exists('Clustering_par1'):
    os.makedirs('Clustering_par1')
plt.savefig('Clustering_par1/elbow1.png',bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
temp = 0
best = 0
from sklearn.metrics import silhouette_score
silhouette_avg = []
K = range(2,15)
for num_clusters in K:
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(ClusterData1s)
    cluster_labels = kmeans.labels_
    temp = silhouette_score(ClusterData1s, cluster_labels)
    silhouette_avg.append(temp)
    max_value = max(silhouette_avg)
    best = silhouette_avg.index(max_value) + 2
    print(silhouette_avg)
    print(best)


In [ ]:
#K-Means Clustering Algorithm
kmeans = KMeans(n_clusters=3)
kmeans.fit(ClusterData1s)
ClusterData['Clustered'] = pd.Series(kmeans.labels_, index=ClusterData.index)
ClusterData['Clustered'].value_counts()

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.scatterplot(x='NNodes', y='TimelimitRaw', hue='Clustered',data=ClusterData,palette="bright")
ax.set_ylabel("Time limit in minutes", fontsize = 14)
ax.set_xlabel("Number of nodes", fontsize = 14)
if not os.path.exists('Clustering_par1'):
    os.makedirs('Clustering_par1')
plt.savefig('Clustering_par1/scatterplot_k-means1.png',bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
fig = px.scatter(ClusterData, x='NNodes', y='TimelimitRaw', color='Clustered')
fig.show()

# Aglomerative Clustering

In [ ]:
temp = 0
best = 0
from sklearn.metrics import silhouette_score
silhouette_avg = []
K = range(2,15)
for num_clusters in K:
    agg = AgglomerativeClustering(n_clusters= num_clusters, affinity='euclidean', linkage='ward')
    agg.fit(ClusterData1s)
    cluster_labels = agg.labels_
    temp = silhouette_score(ClusterData1s, cluster_labels)
    silhouette_avg.append(temp)
    max_value = max(silhouette_avg)
    best = silhouette_avg.index(max_value) + 2
    print(silhouette_avg)
    print(best)

In [ ]:
model = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')
model.fit(ClusterData1s)
ClusterData['Dendcluster'] = pd.Series(model.labels_, index=ClusterData.index)
ClusterData['Dendcluster'].value_counts()

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x='NNodes', y='TimelimitRaw', hue='Dendcluster',data=ClusterData,palette="bright")
if not os.path.exists('Clustering_par1'):
    os.makedirs('Clustering_par1')
plt.savefig('Clustering_par1/agglomerative.png',bbox_inches='tight', dpi=100)
plt.show()

# HDB Scan

### Drop the clustered columns created previously for example clustered, dendcluster

In [ ]:

#ClusterDataa = ClusterData.drop(['JobID','Clustered','Dendcluster','DBcluster'], axis=1)

In [ ]:
ClusterDataa.head()

In [ ]:
DB = hdbscan.HDBSCAN(min_samples =5,cluster_selection_epsilon =0.08,min_cluster_size = 50)
DB.fit_predict(ClusterData1s)
ClusterData['DBcluster'] = pd.Series(DB.labels_, index=ClusterData.index)
ClusterData['DBcluster'].value_counts()

In [ ]:
fig = px.scatter(ClusterData, x='NNodes', y='TimelimitRaw', color='DBcluster',width=2000, height=1000)
fig.show()

In [ ]:
plt.figure(figsize=(15, 5))

palette = sns.color_palette(cc.glasbey, n_colors=57)
ax = sns.scatterplot(x='NNodes', y='TimelimitRaw', hue='DBcluster',data=ClusterData,palette=palette, legend = False)
ax.set_xlabel("Time limit in minutes", fontsize = 14)
ax.set_ylabel("Number of nodes", fontsize = 14)
if not os.path.exists('Clustering_par1'):
    os.makedirs('Clustering_par1')
plt.savefig('Clustering_par1/HDBSCAN.png',bbox_inches='tight', dpi=100)
plt.show()

## Normalization

In [ ]:
ClusterData2s = ClusterData.drop(['JobID','Clustered','Dendcluster','DBcluster'], axis=1)
sc_x2 = MinMaxScaler()
ClusterData2s = sc_x2.fit_transform(ClusterData2s)
#Elbow Method to determine the best K value
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(ClusterData2s)
    distortions.append(kmeanModel.inertia_)
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
ClusterData2s

In [ ]:
temp = 0
best = 0
from sklearn.metrics import silhouette_score
silhouette_avg = []
K = range(2,15)
for num_clusters in K:
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(ClusterData2s)
    cluster_labels = kmeans.labels_
    temp = silhouette_score(ClusterData2s, cluster_labels)
    silhouette_avg.append(temp)
    max_value = max(silhouette_avg)
    best = silhouette_avg.index(max_value) + 2
    print(silhouette_avg)
    print(best)


In [ ]:
#K-Means Clustering Algorithm
kmeans4 = KMeans(n_clusters=2)
kmeans4.fit(ClusterData2s)
ClusterData['Clustered'] = pd.Series(kmeans4.labels_, index=ClusterData.index)
ClusterData['Clustered'].value_counts()

In [ ]:
plt.figure(figsize=(15, 5))
sns.scatterplot(x='NNodes', y='TimelimitRaw', hue='Clustered',data=ClusterData,palette="bright")

# For all the records

In [ ]:
Data = slurm[['JobID','NNodes','TimelimitRaw']]
sns.pairplot(Data)
if not os.path.exists('Clustering_par1'):
    os.makedirs('Clustering_par1')
plt.savefig('Clustering_par1/pairplot2.png',bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
p = sns.scatterplot(x='NNodes', y='TimelimitRaw',data=Data,palette="bright")
p.set_xlabel("Nodes", fontsize = 20)
p.set_ylabel("Timelimit in minutes", fontsize = 16)
if not os.path.exists('Clustering_par1'):
    os.makedirs('Clustering_par1')
plt.savefig('Clustering_par1/data.png',bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
Data.info()

In [ ]:
Data1s = Data.drop(['JobID'], axis=1)
sc_x = StandardScaler()
Data1s = sc_x.fit_transform(Data1s)
#Elbow Method to determine the best K value
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(Data1s)
    distortions.append(kmeanModel.inertia_)
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
Data1s

In [ ]:
temp = 0
best = 0
from sklearn.metrics import silhouette_score
silhouette_avg = []
K = range(2,15)
for num_clusters in K:
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(Data1s)
    cluster_labels = kmeans.labels_
    temp = silhouette_score(Data1s, cluster_labels)
    silhouette_avg.append(temp)
    max_value = max(silhouette_avg)
    best = silhouette_avg.index(max_value) + 2
    print(silhouette_avg)
    print(best)


In [ ]:
#K-Means Clustering Algorithm
kmeansa = KMeans(n_clusters=10)
kmeansa.fit(Data1s)
Data['Clustered'] = pd.Series(kmeansa.labels_, index=Data.index)
Data['Clustered'].value_counts()

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

import matplotlib.pyplot as plt
import seaborn as sns
palette = sns.color_palette(cc.glasbey, n_colors=10)
ax = sns.scatterplot(x='NNodes', y='TimelimitRaw', hue='Clustered',data=Data,palette=palette)
ax.set_ylabel("Time limit in minutes", fontsize = 14)
ax.set_xlabel("Number of nodes", fontsize = 14)
if not os.path.exists('clustering_all'):
    os.makedirs('clustering_all')
plt.savefig('clustering_all/clustering1.png')

# HDB Scan

In [ ]:
DB = hdbscan.HDBSCAN(min_samples = 2,cluster_selection_epsilon = 0.1,min_cluster_size = 50)
DB.fit_predict(Data1s)
Data['DBcluster'] = pd.Series(DB.labels_, index=Data.index)
a = Data['DBcluster'].value_counts()

In [ ]:
a

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10
import colorcet as cc
import matplotlib.pyplot as plt
import seaborn as sns
palette = sns.color_palette(cc.glasbey, n_colors=len(a))
ax = sns.scatterplot(x='NNodes', y='TimelimitRaw', hue='DBcluster',data=Data,palette = palette,legend = False )
ax.set_ylabel("Time limit in minutes", fontsize = 14)
ax.set_xlabel("Number of nodes", fontsize = 14)
if not os.path.exists('clustering_all'):
    os.makedirs('clustering_all')
plt.savefig('clustering_all/clustering3.png',bbox_inches='tight', dpi=100)